In [1]:
from pyspark import SparkConf,SparkContext
import sys
from math import sqrt

In [2]:
def loadMovieNames():
    movieNames={}
    with open("///home/admin/Frank-Kanes/ml-100k/u.item") as f:
        for line in f:
            fields=line.split('|')
            movieNames[int(fields[0])]=fields[1].decode('ascii','ignore')
        return movieNames

In [3]:
nameDict=loadMovieNames()

In [4]:
data=sc.textFile("file:///home/admin/Frank-Kanes/ml-100k/u.data")

In [5]:
ratings=data.map(lambda l : l.split()).map(lambda l : ((int(l[0])) , (int(l[1]) , float(l[2]))))

In [6]:
#every possible movie rating pairs
joinedRatings=ratings.join(ratings)

In [9]:
joinedRatings.take(5)

[(512, ((265, 4.0), (265, 4.0))),
 (512, ((265, 4.0), (23, 4.0))),
 (512, ((265, 4.0), (1, 4.0))),
 (512, ((265, 4.0), (198, 5.0))),
 (512, ((265, 4.0), (318, 5.0)))]

In [10]:
def filterDuplicateRecords((userID,ratings)):
    (movie1,rating1)=ratings[0]
    (movie2,rating2)=ratings[1]
    return movie1 < movie2

In [11]:
#filter out the duplicate records
uniqueJoinedRecords=joinedRatings.filter(filterDuplicateRecords)

In [12]:
uniqueJoinedRecords.take(5)

[(512, ((265, 4.0), (318, 5.0))),
 (512, ((265, 4.0), (1459, 4.0))),
 (512, ((265, 4.0), (313, 3.0))),
 (512, ((265, 4.0), (527, 5.0))),
 (512, ((265, 4.0), (302, 4.0)))]

In [13]:
#key the movie pair
def makePairs((user,ratings)):
    (movie1,rating1)=ratings[0]
    (movie2,rating2)=ratings[1]
    return ((movie1,movie2) , (rating1,rating2))

In [14]:
moviePairs=uniqueJoinedRecords.map(makePairs)

In [15]:
moviePairs.take(5)

[((265, 318), (4.0, 5.0)),
 ((265, 1459), (4.0, 4.0)),
 ((265, 313), (4.0, 3.0)),
 ((265, 527), (4.0, 5.0)),
 ((265, 302), (4.0, 4.0))]

In [17]:
moviePairRatings=moviePairs.groupByKey()

In [18]:
moviePairRatings.take(5)

[((197, 1097), <pyspark.resultiterable.ResultIterable at 0x7f3f744dad50>),
 ((42, 364), <pyspark.resultiterable.ResultIterable at 0x7f3f744dad90>),
 ((773, 1409), <pyspark.resultiterable.ResultIterable at 0x7f3f744dae10>),
 ((273, 617), <pyspark.resultiterable.ResultIterable at 0x7f3f744dae50>),
 ((372, 974), <pyspark.resultiterable.ResultIterable at 0x7f3f744dae90>)]

In [20]:
def computeCosineSimilarity(ratingPairs):
    numPairs=0
    sum_xx=sum_yy=sum_xy=0
    for ratingX,ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingXY * ratingXY
            
    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

In [21]:
moviePairSimilarities=moviePairRatings.mapValues(computeCosineSimilarity).cache()

In [22]:
scoreThreshold = 0.97
coOccurenceThreshold = 50

movieID = 80

    # Filter for movies with this sim that are "good" as defined by
    # our quality thresholds above
filteredResults = moviePairSimilarities.filter(lambda((pair,sim)): \
        (pair[0] == movieID or pair[1] == movieID) \
        and sim[0] > scoreThreshold and sim[1] > coOccurenceThreshold)

In [24]:
results = filteredResults.map(lambda((pair,sim)): (sim, pair)).sortByKey(ascending = False).take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 16.0 failed 1 times, most recent failure: Lost task 1.0 in stage 16.0 (TID 18, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "/usr/lib/spark/python/pyspark/rdd.py", line 1983, in <lambda>
    map_values_fn = lambda kv: (kv[0], f(kv[

In [ ]:
print("Top 10 similar movies for " + nameDict[movieID])
for result in results:
    (sim, pair) = result
        # Display the similarity result that isn't the movie we're looking at
    similarMovieID = pair[0]
    if (similarMovieID == movieID):
        similarMovieID = pair[1]
    print(nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1]))